In [36]:
!pip install transformers peft torch sacremoses


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import sacremoses

c:\Users\Muhammad Ahmad\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

# Load base model (BioGPT)
base_model_name = "microsoft/biogpt"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Set the adapter path (folder containing saved files)
adapter_path = "C:/Users/Muhammad Ahmad/Downloads/BioGPT Fine of FYP/model"

In [5]:
# Load base model on CPU
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, torch_dtype=torch.float32).to("cpu")

# Load the LoRA adapter
model = PeftModel.from_pretrained(base_model, adapter_path)
model = model.to("cpu")  # Ensure model runs on CPU

In [6]:


# Function to generate responses
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
    output = model.generate(
        **inputs, 
        max_length=200, 
        temperature=0.8,  # Increase randomness
        top_p=0.9,  # Nucleus sampling for diverse output
        do_sample=True  # Enable sampling instead of greedy decoding
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


# Example usage
prompt = "Child is 1 years old, height 77 cm, weight 10 kg, sex Female. Last week's diet: High-calorie snacks and processed food..\nAnswer:"
response = generate_response(prompt)
#print(response)


In [7]:
def extract_and_format_answer(response):
  """
  Extracts the answer and formats it into separate points.

  Args:
    response: The input string containing the answer.
  """
  answer_index = response.find("Answer:")
  if answer_index != -1:
    answer_text = response[answer_index + len("Answer:"):].strip()

    risk_level_index = answer_text.find("Malnutrition Risk Level:")
    deficiency_index = answer_text.find("Key Deficiency Noticed:")
    advice_index = answer_text.find("Immediate Advice:")

    if risk_level_index != -1 and deficiency_index != -1 and advice_index != -1:
      risk_level = answer_text[risk_level_index:deficiency_index].strip()
      deficiency = answer_text[deficiency_index:advice_index].strip()
      advice = answer_text[advice_index:].strip()

      print(risk_level)
      print(deficiency)
      print(advice)
    else:
      print(answer_text) # Print as is if keywords are missing or out of order.
  else:
    print("Answer: keyword not found.")

# Example usage with your provided paragraph:

extract_and_format_answer(response)


Malnutrition Risk Level: Risk of Overweight (Based on BMI-for-age percentile and growth chart analysis)
Key Deficiency Noticed: - Potential deficiency of fiber and essential vitamins. - Increased risk of metabolic disorders due to excessive high-calorie intake.
Immediate Advice: - Reduce consumption of processed foods, sugary snacks, and high-fat meals. - Encourage daily physical activity, such as outdoor play and simple exercises. - Introduce fiber-rich foods such as fruits, vegetables, and whole grains. - Ensure proper hydration and limit sugary drinks. - Establish structured meal times with balanced nutrition. - Establish structured meal times with balanced nutrition. - Ensure proper hydration and limit sugary drinks. - Establish structured meal size and phase-specific meals for daily physical
